In [14]:
from parsita import *
from parsita.util import constant
import json

# Conversion functions
def formatannotations(annotations):
    return {ant[0]: ant[1] for ant in annotations}

def formatgame(game):
    return {
        'moves': game[0],
        'outcome': game[1]
    }

def formatentry(entry):
    return {'annotations': entry[0], 'game': entry[1]}

def handleoptional(optionalmove):
    
    if len(optionalmove) > 0:
        return optionalmove[0]
    
    else:
        return None
    
# PGN Grammar
quote = lit(r'"')
tag = reg(r'[\u0021-\u0021\u0023-\u005A\u005E-\u007E]+')
string = reg(r'[\u0020-\u0021\u0023-\u005A\u005E-\U0010FFFF]+')
whitespace = lit(' ') | lit('\n')
annotation = '[' >> (tag) << ' ' & (quote >> string << quote) << ']'
annotations = repsep(annotation, '\n') > formatannotations
nullmove = lit('--') # Illegal move rarely used in annotations
longcastle = reg(r'O-O-O[+#]?')
castle = reg(r'O-O[+#]?')
regularmove = reg(r'[a-h1-8NBRQKx\+#=]+') # Matches more than just chess moves
move = regularmove | longcastle | castle | nullmove
movenumber = (reg(r'[0-9]+') << '.' << whitespace) > int
turn = movenumber & (move << whitespace) & (opt(move << whitespace) > handleoptional)
draw = lit('1/2-1/2')
white = lit('1-0')
black = lit('0-1')
outcome = draw | white | black
game = (rep(turn) & outcome) > formatgame
entry = ((annotations << rep(whitespace)) & (game << rep(whitespace))) > formatentry
file = rep(entry)

# Parse the file
with open('all.pgn', 'r', encoding="utf8", errors='ignore') as f:
    parsedoutput = file.parse(f.read()).or_die()

ParseError: Expected ' ' or '\n' or r'[0-9]+' or '1/2-1/2' or '1-0' or '0-1' but found '\t   '
Line 13, character 1

	   1.	d4 	d5 
^              

In [13]:
print(parsedoutput)

Failure("Expected ' ' or '\\n' or r'[0-9]+' or '1/2-1/2' or '1-0' or '0-1' but found '\\t   '\nLine 13, character 1\n\n\t   1.\td4 \td5 \n^              ")
